## Extracting "Common Names" section of the various medicines from Drugs.com Website

In [ ]:
# Imorting Modules
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import pathlib
from pandas import DataFrame

In [ ]:
url = 'https://www.drugs.com/npc/' 
url2 = 'https://www.drugs.com'

In [ ]:
#Requests module use to data from given url
source=requests.get(url)

# BeautifulSoup is used for getting HTML structure from requests response.
soup=BeautifulSoup(source.text,'html')
body = soup.body.findAll ('div',{'id': 'a2z'})

In [ ]:
#Extracting the links present on the web page
medicines = {}
links = []
a = 0
for tags in body:
    for t in tags.find_all('a', href=True):
        links.append(t.get('href'))
        medicines[a] = t.get_text('href')
        a += 1
print(medicines)
print(links) 
keys = list(medicines.values())

{0: '5-Hydroxytryptophan', 1: 'Acai', 2: 'Acerola', 3: 'Acetyl-L-Carnitine', 4: 'Acidophilus', 5: 'Aconite', 6: 'Adrenal Extract', 7: 'African Mango', 8: 'Alfalfa', 9: 'Aloe', 10: 'Alpha-Lipoic Acid', 11: 'Angelica', 12: 'Anise', 13: 'Aortic Extract', 14: 'Arabinoxylane', 15: 'Arginine', 16: 'Arnica', 17: 'Artichoke', 18: 'Ashwagandha', 19: 'Astragalus', 20: 'Barberry', 21: 'Barley Grass', 22: 'Bayberry', 23: 'Bee Pollen', 24: 'Beta Glucans', 25: 'Beta-Carotene', 26: 'Beta-Sitosterol', 27: 'Betaine Hydrochloride', 28: 'Betel Nut', 29: 'Betony', 30: 'Bilberry', 31: 'Biotin', 32: 'Bitter Melon', 33: 'Bitter Orange', 34: 'Black Cohosh', 35: 'Bladderwrack', 36: 'Bloodroot', 37: 'Boldo', 38: 'Boneset', 39: 'Borage', 40: 'Boron', 41: 'Bovine Colostrum', 42: 'Brahmi', 43: 'Branched-Chain Amino Acids', 44: "Brewer's Yeast", 45: 'Bromelain', 46: 'Buchu', 47: 'Bugleweed', 48: 'Bupleurum', 49: 'Burdock', 50: "Butcher's Broom", 51: 'Butterbur', 52: 'Calanolide A', 53: 'Calendula', 54: 'Caprylic Ac

In [ ]:
# function for extraction of required data using HTML tags
def alternate_names(soups_):
    common_names = ''
    for tags in soups_.find_all("div",{"class": "list synlist"}) :
        for t in tags.find_all("li"):
            common_names  = common_names + '\n' + t.get_text()
            
    if len(common_names) != 0 :
        return(common_names)
    else :
        return((alternate_names_2(soups_)))
    

In [ ]:
# function for extraction of required data by converting the contents as text and searching the required section
def alternate_names_2 (soups_):
    
    text = "Common Name(s)"
    
    #Extracting the text present in class = "contentBox"
    for tags in soups_.find_all("div",{"class": "contentBox"}):
        string = (tags.get_text())
        
    pos = string.find(text)
    
    if pos == -1:
        return(alternate_names_3 (soups_))
    else:
        new_string = string[pos+len(text)+1:]

        end = new_string.find("What is")

        desired_output =  (new_string[:end]).rstrip()
        return(desired_output)

In [ ]:
# function for extraction of required data by converting the contents as text and searching the required section
def alternate_names_3 (soups_):
    
    text = "Common names:"
    
    #Extracting the text present in class = "contentBox"
    for tags in soups_.find_all("div",{"class": "contentBox"}):
        string = (tags.get_text())
        
    pos = string.find(text)
    
    if pos == -1:
        return(None)
    else:
        new_string = string[pos+len(text)+1:]

        end = new_string.find("Efficacy rating")

        desired_output =  (new_string[:end]).rstrip()
        return(desired_output)

In [ ]:
start = 0
names = {}
# First checking the file exist or not. If it exists, checking whether it is empty or not
name = 'Common_Name.csv'
file_name = pathlib.Path(name)
if file_name.exists ():
    print ("File exist.")
    try :
        print("Reading the file now")
        df = pd.read_csv(name, index_col = 0)

        start = (len(df) - 1)
    except:
        print("File is empty")
        
    
else:
    print ("File does not exist\n", "Creating the file")
    file = open(name,"w+")
    print("File created successfully with filename - ",name)
    
print("No.of links are - ", len(links))

File does not exist
 Creating the file
File created successfully with filename -  Common_Name.csv
No.of links are -  259


In [ ]:
suc = 0
fail = 0
failed_links = []
# Iterating through the links and extracting data
for i in range(start,len(links)) :
    print (" Extracting data for -", keys[i])    
    link_url = url2 + links[i]    
    source=requests.get(link_url)

    # BeautifulSoup is used for getting HTML structure from requests response.
    soups=BeautifulSoup(source.text,'html')
    common_names = alternate_names (soups)
    # Avoiding errors
    try :
        if len(common_names) != 0 or common_name is None:
            print("Success in extracting data for - ",keys[i])
            print("Copying the data from the web........")
            names[keys[i]] = common_names
            print("DONE")
            print("*"*50)
            suc += 1


        else :
            print("No data was extracted for - ", keys[i])
            names[keys[i]] = None
            print("*"*50)
            fail += 1
            failed_links.append(link_url)
    except :
        
        print("No data was extracted for - ", keys[i])
        names[keys[i]] = None
        print("*"*50)
        fail += 1
        failed_links.append(link_url)
        
# Let's see the links from which the data extraction was a success/unsuccess
print("No.of links from which data was extracted succesfully-", suc)
print("No.of links from which data extraction was unsuccesfull-", fail)
print("*"*20,'PROCESS COMPLETED',"*"*20)

 Extracting data for - 5-Hydroxytryptophan
Success in extracting data for -  5-Hydroxytryptophan
Copying the data from the web........
DONE
**************************************************
 Extracting data for - Acai
Success in extracting data for -  Acai
Copying the data from the web........
DONE
**************************************************
 Extracting data for - Acerola
Success in extracting data for -  Acerola
Copying the data from the web........
DONE
**************************************************
 Extracting data for - Acetyl-L-Carnitine
Success in extracting data for -  Acetyl-L-Carnitine
Copying the data from the web........
DONE
**************************************************
 Extracting data for - Acidophilus
No data was extracted for -  Acidophilus
**************************************************
 Extracting data for - Aconite
Success in extracting data for -  Aconite
Copying the data from the web........
DONE
*************************************************

In [ ]:
failed_links

['https://www.drugs.com/npc/acidophilus.html',
 'https://www.drugs.com/npc/boneset.html',
 'https://www.drugs.com/npc/calanolide-a.html',
 'https://www.drugs.com/npc/hops.html',
 'https://www.drugs.com/npc/jojoba.html',
 'https://www.drugs.com/npc/l-arginine.html',
 'https://www.drugs.com/npc/saffron.html',
 'https://www.drugs.com/npc/shark-derivatives.html',
 'https://www.drugs.com/npc/shark-liver-oil.html',
 'https://www.drugs.com/npc/sod.html']

In [ ]:
# Appending the new values
df_temp = DataFrame(list(names.items()),columns = ['Medicines','Common_Names']) 
try:
    result = pd.concat([df,df_temp], ignore_index = True)
    result
except :
    result = df_temp
result

,Medicines,Common_Names
0,5-Hydroxytryptophan,\n2-Amino-3-(5-Hydroxy-1H-Indol-3-yl)Propanoic...
1,Acai,\n\nEuterpe oleracea\n\nAcai Berries\nAcai Ber...
2,Acerola,"Acerola also is known as Barbados cherry, West..."
3,Acetyl-L-Carnitine,\nAcetyl Carnitine\nAcetyl L-Carnitine Hydroch...
4,Acidophilus,None
...,...,...
254,Yerba Santa,"Yerba santa also is known as eriodictyon, tarw..."
255,Yogurt,Yogurt
256,Yohimbe,\n11-Hydroxy Yohimbine\n\nCorynanthe johimbi\n...
257,Yucca,"Yucca also is known as Spanish bayonet, Our Lo..."


In [ ]:
result = result.drop_duplicates()
#writing to the file
result.to_csv(name)

In [ ]:
result

,Medicines,Common_Names
0,5-Hydroxytryptophan,\n2-Amino-3-(5-Hydroxy-1H-Indol-3-yl)Propanoic...
1,Acai,\n\nEuterpe oleracea\n\nAcai Berries\nAcai Ber...
2,Acerola,"Acerola also is known as Barbados cherry, West..."
3,Acetyl-L-Carnitine,\nAcetyl Carnitine\nAcetyl L-Carnitine Hydroch...
4,Acidophilus,None
...,...,...
254,Yerba Santa,"Yerba santa also is known as eriodictyon, tarw..."
255,Yogurt,Yogurt
256,Yohimbe,\n11-Hydroxy Yohimbine\n\nCorynanthe johimbi\n...
257,Yucca,"Yucca also is known as Spanish bayonet, Our Lo..."
